<a href="https://colab.research.google.com/github/nagarajabhishek3/Google-colab-files/blob/main/budget_day_backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import yfinance as yf

# ---------- 1) Choose instrument ----------
# NIFTY 50 on Yahoo Finance:
SYMBOL = "^NSEI"     # change to "RELIANCE.NS", "HDFCBANK.NS", etc.

# ---------- 2) Budget presentation dates (last ~15 years) ----------
# Includes interim/full where applicable (2014, 2019, 2024).
budget_dates = [
    "2011-02-28",
    "2012-03-16",
    "2013-02-28",
    "2014-02-17",  # Interim
    "2014-07-10",  # Full
    "2015-02-28",
    "2016-02-29",
    "2017-02-01",
    "2018-02-01",
    "2019-02-01",  # Interim
    "2019-07-05",  # Full
    "2020-02-01",
    "2021-02-01",
    "2022-02-01",
    "2023-02-01",
    "2024-02-01",  # Interim (election year)
    "2024-07-23",  # Full
    "2025-02-01",
]

budget_dates = pd.to_datetime(budget_dates)

# ---------- 3) Download daily OHLC ----------
start = (budget_dates.min() - pd.Timedelta(days=30)).strftime("%Y-%m-%d")
end   = (budget_dates.max() + pd.Timedelta(days=30)).strftime("%Y-%m-%d")

df = yf.download(SYMBOL, start=start, end=end, interval="1d", auto_adjust=False, progress=False)
if df.empty:
    raise RuntimeError("No data returned. Try a different SYMBOL or check internet access.")

# Ensure Date index is clean and sorted
df = df.sort_index()
trading_days = df.index

def next_trading_day(d: pd.Timestamp) -> pd.Timestamp:
    """If d is not a trading day, return the next trading day after d."""
    # searchsorted gives insertion position to keep sorted order
    pos = trading_days.searchsorted(d)
    if pos >= len(trading_days):
        return pd.NaT
    # If exact date exists use it, else use next available
    if trading_days[pos] == d:
        return d
    return trading_days[pos]

def prev_trading_day(d: pd.Timestamp) -> pd.Timestamp:
    """Return the previous trading day before d (strictly before)."""
    pos = trading_days.searchsorted(d)
    if pos <= 0:
        return pd.NaT
    # if d is a trading day, previous is pos-1; if not, still pos-1
    return trading_days[pos - 1]

rows = []
for bd in budget_dates:
    actual_day = next_trading_day(bd)
    if pd.isna(actual_day):
        continue

    prev_day = prev_trading_day(actual_day)
    if pd.isna(prev_day):
        continue

    o = float(df.loc[actual_day, "Open"])
    h = float(df.loc[actual_day, "High"])
    l = float(df.loc[actual_day, "Low"])
    c = float(df.loc[actual_day, "Close"])
    prev_close = float(df.loc[prev_day, "Close"])

    rows.append({
        "budget_date": bd.date(),
        "used_trading_day": actual_day.date(),
        "is_holiday_shifted": actual_day.date() != bd.date(),
        "prev_trading_day": prev_day.date(),
        "prev_close": prev_close,
        "open": o,
        "high": h,
        "low": l,
        "close": c,
        # Useful return metrics:
        "gap_%": (o / prev_close - 1) * 100,
        "intraday_%": (c / o - 1) * 100,
        "close_to_prevclose_%": (c / prev_close - 1) * 100,
        "range_%": ((h - l) / prev_close) * 100,
    })

out = pd.DataFrame(rows).sort_values("budget_date")

# ---------- 4) Save results ----------
out.to_csv("budget_day_backtest.csv", index=False)

# ---------- 5) Quick summary ----------
summary = out[["gap_%", "intraday_%", "close_to_prevclose_%", "range_%"]].describe()
print(out.head(10))
print("\nSummary:\n", summary)
print("\nSaved: budget_day_backtest.csv")


  budget_date used_trading_day  is_holiday_shifted prev_trading_day  \
0  2011-02-28       2011-02-28               False       2011-02-25   
1  2012-03-16       2012-03-16               False       2012-03-15   
2  2013-02-28       2013-02-28               False       2013-02-27   
3  2014-02-17       2014-02-18                True       2014-02-14   
4  2014-07-10       2014-07-10               False       2014-07-09   
5  2015-02-28       2015-03-02                True       2015-02-27   
6  2016-02-29       2016-02-29               False       2016-02-26   
7  2017-02-01       2017-02-01               False       2017-01-31   
8  2018-02-01       2018-02-01               False       2018-01-31   
9  2019-02-01       2019-02-01               False       2019-01-31   

     prev_close          open          high           low         close  \
0   5303.549805   5330.149902   5477.000000   5308.600098   5333.250000   
1   5380.500000   5380.350098   5445.649902   5305.000000   5317.899

/tmp/ipython-input-3863007556.py:74: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  o = float(df.loc[actual_day, "Open"])
/tmp/ipython-input-3863007556.py:75: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  h = float(df.loc[actual_day, "High"])
/tmp/ipython-input-3863007556.py:76: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  l = float(df.loc[actual_day, "Low"])
/tmp/ipython-input-3863007556.py:77: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  c = float(df.loc[actual_day, "Close"])
/tmp/ipython-input-3863007556.py:78: FutureWarning: Calling float on a single element Series is deprecated and will rais

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np

# -----------------------------
# 1) Configuration
# -----------------------------
SYMBOL = "^NSEI"   # NIFTY 50
LOOKBACK_DAYS = 5 # 1 trading week

budget_dates = pd.to_datetime([
    "2011-02-28", "2012-03-16", "2013-02-28",
    "2014-02-17", "2014-07-10",
    "2015-02-28", "2016-02-29",
    "2017-02-01", "2018-02-01",
    "2019-02-01", "2019-07-05",
    "2020-02-01", "2021-02-01",
    "2022-02-01", "2023-02-01",
    "2024-02-01", "2024-07-23",
    "2025-02-01"
])

# -----------------------------
# 2) Download price data
# -----------------------------
start = (budget_dates.min() - pd.Timedelta(days=30)).strftime("%Y-%m-%d")
end   = (budget_dates.max() + pd.Timedelta(days=5)).strftime("%Y-%m-%d")

df = yf.download(SYMBOL, start=start, end=end, progress=False)
df = df.sort_index()

# -----------------------------
# 3) Utility functions
# -----------------------------
def prev_trading_day(d):
    pos = df.index.searchsorted(d)
    # Ensure pos is valid and the previous day is within the DataFrame
    if pos == 0: # If d is before or is the first trading day, no previous day
        return None
    prev_day_index = df.index[pos - 1]
    # Check if the found previous day is actually before `d`
    if prev_day_index < d:
        return prev_day_index
    return None

# -----------------------------
# 4) Pre-Budget Trend Analysis
# -----------------------------
rows = []

for bd in budget_dates:
    budget_td = prev_trading_day(bd)
    if budget_td is None:
        continue

    # Extract the 'Close' prices for the SYMBOL, assuming MultiIndex columns
    # as indicated by the kernel state's df variable content (e.g., ('Close', '^NSEI')).
    close_prices_window = df.loc[:budget_td, ('Close', SYMBOL)].tail(LOOKBACK_DAYS + 1)

    if len(close_prices_window) < LOOKBACK_DAYS + 1:
        continue

    # These will now be scalar floats as close_prices_window is a Series
    start_price = close_prices_window.iloc[0]
    end_price   = close_prices_window.iloc[-1]

    # `close_prices_window` is a Series, so `pct_change()` will return a Series.
    returns = close_prices_window.pct_change().dropna()

    # `total_return` and `volatility` will now be scalar floats
    total_return = (end_price / start_price - 1) * 100
    volatility = returns.std() * np.sqrt(252) * 100

    # Trend classification
    if total_return > 1:
        trend = "Uptrend"
    elif total_return < -1:
        trend = "Downtrend"
    else:
        trend = "Sideways"

    rows.append({
        "budget_date": bd.date(),
        "pre_budget_return_%": round(total_return, 2),
        "pre_budget_vol_%": round(volatility, 2),
        "pre_budget_trend": trend
    })

pre_budget_df = pd.DataFrame(rows)

# -----------------------------
# 5) Save & preview
# -----------------------------
pre_budget_df.to_csv("pre_budget_1week_trend.csv", index=False)
print(pre_budget_df)
print("\nSaved: pre_budget_1week_trend.csv")

   budget_date  pre_budget_return_%  pre_budget_vol_% pre_budget_trend
0   2011-02-28                -2.85             27.11        Downtrend
1   2012-03-16                 3.07             21.72          Uptrend
2   2013-02-28                -2.46             15.98        Downtrend
3   2014-02-17                -0.24             12.89         Sideways
4   2014-07-10                -1.81             16.82        Downtrend
5   2015-02-28                 0.12             17.98         Sideways
6   2016-02-29                -2.51             17.12        Downtrend
7   2017-02-01                 2.02             14.48          Uptrend
8   2018-02-01                -0.51              7.27         Sideways
9   2019-02-01                -0.17             16.76         Sideways
10  2019-07-05                 0.89              6.51         Sideways
11  2020-02-01                -2.34             10.12        Downtrend
12  2021-02-01                -6.55              6.09        Downtrend
13  20

/tmp/ipython-input-3238848561.py:29: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(SYMBOL, start=start, end=end, progress=False)


In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np

# -----------------------------
# 1) Config
# -----------------------------
SYMBOL = "^NSEI"
LOOKBACK_DAYS = 5

budget_dates = pd.to_datetime([
    "2011-02-28", "2012-03-16", "2013-02-28",
    "2014-02-17", "2014-07-10",
    "2015-02-28", "2016-02-29",
    "2017-02-01", "2018-02-01",
    "2019-02-01", "2019-07-05",
    "2020-02-01", "2021-02-01",
    "2022-02-01", "2023-02-01",
    "2024-02-01", "2024-07-23",
    "2025-02-01"
])

# -----------------------------
# 2) Download data
# -----------------------------
start = (budget_dates.min() - pd.Timedelta(days=30)).strftime("%Y-%m-%d")
end   = (budget_dates.max() + pd.Timedelta(days=5)).strftime("%Y-%m-%d")

df = yf.download(SYMBOL, start=start, end=end, progress=False)
df = df.sort_index()

# -----------------------------
# 3) Helpers
# -----------------------------
def next_trading_day(d):
    pos = df.index.searchsorted(d)
    if pos < len(df.index):
        return df.index[pos]
    return None

def prev_trading_day(d):
    pos = df.index.searchsorted(d)
    return df.index[pos - 1] if pos > 0 else None

# -----------------------------
# 4) Build dataset
# -----------------------------
rows = []

for bd in budget_dates:
    budget_td = next_trading_day(bd)
    prev_td   = prev_trading_day(budget_td)

    if budget_td is None or prev_td is None:
        continue

    # Pre-budget window
    pre_window = df.loc[:prev_td].tail(LOOKBACK_DAYS + 1)
    if len(pre_window) < LOOKBACK_DAYS + 1:
        continue

    pre_return = (pre_window.iloc[-1]["Close"] /
                  pre_window.iloc[0]["Close"] - 1) * 100

    # Budget-day return
    budget_close_ret = (df.loc[budget_td]["Close"] /
                        df.loc[prev_td]["Close"] - 1) * 100

    rows.append({
        "budget_date": bd.date(),
        "pre_budget_1w_return_%": round(pre_return, 2),
        "budget_day_close_%": round(budget_close_ret, 2)
    })

corr_df = pd.DataFrame(rows)

# -----------------------------
# 5) Correlation analysis
# -----------------------------
pearson_corr = corr_df["pre_budget_1w_return_%"].corr(
    corr_df["budget_day_close_%"]
)

print(corr_df)
print(f"\nPearson Correlation: {round(pearson_corr, 3)}")

# Save for inspection
corr_df.to_csv("pre_budget_vs_budget_day_correlation.csv", index=False)
print("\nSaved: pre_budget_vs_budget_day_correlation.csv")


   budget_date               pre_budget_1w_return_%  \
0   2011-02-28  Ticker
^NSEI   -2.85
dtype: float64   
1   2012-03-16  Ticker
^NSEI    3.07
dtype: float64   
2   2013-02-28  Ticker
^NSEI   -2.46
dtype: float64   
3   2014-02-17  Ticker
^NSEI   -0.24
dtype: float64   
4   2014-07-10  Ticker
^NSEI   -1.81
dtype: float64   
5   2015-02-28  Ticker
^NSEI    0.12
dtype: float64   
6   2016-02-29  Ticker
^NSEI   -2.51
dtype: float64   
7   2017-02-01  Ticker
^NSEI    2.02
dtype: float64   
8   2018-02-01  Ticker
^NSEI   -0.51
dtype: float64   
9   2019-02-01  Ticker
^NSEI   -0.17
dtype: float64   
10  2019-07-05  Ticker
^NSEI    0.89
dtype: float64   
11  2020-02-01  Ticker
^NSEI   -2.34
dtype: float64   
12  2021-02-01  Ticker
^NSEI   -6.55
dtype: float64   
13  2022-02-01  Ticker
^NSEI   -1.57
dtype: float64   
14  2023-02-01  Ticker
^NSEI   -2.52
dtype: float64   
15  2024-02-01  Ticker
^NSEI    2.29
dtype: float64   
16  2024-07-23  Ticker
^NSEI    0.03
dtype: float64   
17  2025-0

/tmp/ipython-input-763496118.py:29: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(SYMBOL, start=start, end=end, progress=False)
/tmp/ipython-input-763496118.py:80: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pearson_corr = corr_df["pre_budget_1w_return_%"].corr(


In [ ]:
import pandas as pd
import yfinance as yf

# ---------------------------------
# 1) Config
# ---------------------------------
SECTORS = {
    "PSU_Banks_proxy": "^NSEBANK",
    "Infra": "^CNXINFRA"
}

budget_dates = pd.to_datetime([
    "2011-02-28", "2012-03-16", "2013-02-28",
    "2014-02-17", "2014-07-10",
    "2015-02-28", "2016-02-29",
    "2017-02-01", "2018-02-01",
    "2019-02-01", "2019-07-05",
    "2020-02-01", "2021-02-01",
    "2022-02-01", "2023-02-01",
    "2024-02-01", "2024-07-23",
    "2025-02-01"
])

# ---------------------------------
# 2) Helper functions
# ---------------------------------
def next_trading_day(df, d):
    pos = df.index.searchsorted(d)
    return df.index[pos] if pos < len(df.index) else None

def prev_trading_day(df, d):
    pos = df.index.searchsorted(d)
    return df.index[pos - 1] if pos > 0 else None

# ---------------------------------
# 3) Run analysis for each sector
# ---------------------------------
all_results = []

for sector, symbol in SECTORS.items():

    start = (budget_dates.min() - pd.Timedelta(days=30)).strftime("%Y-%m-%d")
    end   = (budget_dates.max() + pd.Timedelta(days=30)).strftime("%Y-%m-%d")

    df = yf.download(symbol, start=start, end=end, progress=False)
    df = df.sort_index()

    # FIX: Flatten MultiIndex columns (CRITICAL)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0) # Changed from -1 to 0

    for bd in budget_dates:
        trade_day = next_trading_day(df, bd)
        prev_day  = prev_trading_day(df, trade_day)

        if trade_day is None or prev_day is None:
            continue

        prev_close = df.loc[prev_day, "Close"]
        o = df.loc[trade_day, "Open"]
        h = df.loc[trade_day, "High"]
        l = df.loc[trade_day, "Low"]
        c = df.loc[trade_day, "Close"]

        all_results.append({
            "sector": sector,
            "budget_date": bd.date(),
            "used_trading_day": trade_day.date(),
            "is_holiday_shifted": trade_day.date() != bd.date(),
            "prev_close": round(prev_close, 2),
            "open": round(o, 2),
            "high": round(h, 2),
            "low": round(l, 2),
            "close": round(c, 2),
            "gap_%": round((o / prev_close - 1) * 100, 2),
            "intraday_%": round((c / o - 1) * 100, 2),
            "close_to_prevclose_%": round((c / prev_close - 1) * 100, 2),
            "range_%": round(((h - l) / prev_close) * 100, 2)
        })

# ---------------------------------
# 4) Output
# ---------------------------------
sector_df = pd.DataFrame(all_results)
sector_df.to_csv("sector_budget_day_performance.csv", index=False)

print(sector_df.head())
print("\nSaved: sector_budget_day_performance.csv")

/tmp/ipython-input-944356049.py:45: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start, end=end, progress=False)
/tmp/ipython-input-944356049.py:45: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start, end=end, progress=False)


            sector budget_date used_trading_day  is_holiday_shifted  \
0  PSU_Banks_proxy  2011-02-28       2011-02-28               False   
1  PSU_Banks_proxy  2012-03-16       2012-03-16               False   
2  PSU_Banks_proxy  2013-02-28       2013-02-28               False   
3  PSU_Banks_proxy  2014-02-17       2014-02-18                True   
4  PSU_Banks_proxy  2014-07-10       2014-07-10               False   

   prev_close      open      high       low     close  gap_%  intraday_%  \
0    10435.73  10512.08  10744.72  10364.23  10435.23   0.73       -0.73   
1    10594.88  10607.63  10832.17  10347.93  10391.23   0.12       -2.04   
2    11930.36  12006.86  12041.31  11401.07  11487.22   0.64       -4.33   
3    10203.63  10324.23  10618.73  10296.13  10575.08   1.18        2.43   
4    14930.93  14939.83  15370.87  14621.78  14821.53   0.06       -0.79   

   close_to_prevclose_%  range_%  
0                 -0.00     3.65  
1                 -1.92     4.57  
2          